<a href="https://colab.research.google.com/github/leduynguyen/soundClassification/blob/master/abnormal_detection_gausian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Upload dataset
* Sample data is located at: https://drive.google.com/open?id=1GTo0lrT9nioY3MnbdbI4mKX9Gso-tO6S

In [0]:
from google.colab import files
uploaded = files.upload()

* Import libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import iplot
from scipy.io import loadmat

* Initial dataset

In [0]:
mat = loadmat("sampleData.mat")
X = mat["X"]
Xval = mat["Xval"]
yval = mat["yval"]

* Visualize the data

In [0]:
plt.scatter(X[:,0],X[:,1],marker="x")
plt.xlim(0,30)
plt.ylim(0,30)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")

* To estimate parameters (mean and variance) for the Gaussian model

In [0]:
def estimateGaussian(X):
    """
     This function estimates the parameters of a Gaussian distribution using 
     the data in X
    """
    #compute mean
    mu = np.mean(X, axis=0)
    # compute sigma
    var = np.cov(X.T)
    var=np.diag(var)
    
    return mu,var

In [0]:
mu, sigma2 = estimateGaussian(X)

* Multivariate Gaussian Distribution

In [0]:
def multivariateGaussian(X, mu, sigma2):
    """
    Computes the probability density function of the multivariate 
    gaussian distribution.
    """
    k = len(mu)
    
    sigma2=np.diag(sigma2)
    X = X - mu.T
    p = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma2)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma2) * X,axis=1))
    return p

In [0]:
p = multivariateGaussian(X, mu, sigma2)

* Visualize the fit

In [0]:
plt.figure(figsize=(8,6))
plt.scatter(X[:,0],X[:,1],marker="x")
X1,X2 = np.meshgrid(np.linspace(0,35,num=70),np.linspace(0,35,num=70))
p2 = multivariateGaussian(np.hstack((X1.flatten()[:,np.newaxis],X2.flatten()[:,np.newaxis])), mu, sigma2)
contour_level = 10**np.array([np.arange(-20,0,3,dtype=np.float)]).T
#plt.contour(X1,X2,p2[:,np.newaxis].reshape(X1.shape),contour_level)
plt.xlim(0,35)
plt.ylim(0,35)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")

* Select the thershold

In [0]:
def selectThreshold(yval, pval):
    """
    Find the best threshold (epsilon) to use for selecting outliers
    """
    best_epi = 0
    best_F1 = 0
    
    stepsize = (max(pval) -min(pval))/1000
    epi_range = np.arange(pval.min(),pval.max(),stepsize)
    for epi in epi_range:
        predictions = (pval<epi)[:,np.newaxis]
        tp = np.sum(predictions[yval==1]==1)
        fp = np.sum(predictions[yval==0]==1)
        fn = np.sum(predictions[yval==1]==0)
        
        # compute precision, recall and F1
        prec = tp/(tp+fp)
        rec = tp/(tp+fn)
        
        F1 = (2*prec*rec)/(prec+rec)
        
        if F1 > best_F1:
            best_F1 =F1
            best_epi = epi
        
    return best_epi, best_F1

In [0]:
pval = multivariateGaussian(Xval, mu, sigma2)
epsilon, F1 = selectThreshold(yval, pval)
print("Best epsilon found using cross-validation:",epsilon)
print("Best F1 on Cross Validation Set:",F1)

* Visualize the anomalies

In [0]:
plt.figure(figsize=(8,6))

# plot the data
plt.scatter(X[:,0],X[:,1],marker="x")

# potting of contour
X1,X2 = np.meshgrid(np.linspace(0,35,num=70),np.linspace(0,35,num=70))
p2 = multivariateGaussian(np.hstack((X1.flatten()[:,np.newaxis],X2.flatten()[:,np.newaxis])), mu, sigma2)
contour_level = 10**np.array([np.arange(-20,0,3,dtype=np.float)]).T
#plt.contour(X1,X2,p2[:,np.newaxis].reshape(X1.shape),contour_level)

# Circling of anomalies
outliers = np.nonzero(p<epsilon)[0]
plt.scatter(X[outliers,0],X[outliers,1],marker ="o",facecolor="none",edgecolor="r",s=70)

plt.xlim(0,35)
plt.ylim(0,35)
plt.xlabel("Latency (ms)")
plt.ylabel("Throughput (mb/s)")